In [84]:
import math
import numpy as np
import matplotlib.pyplot as plt

from my_helpers import *

# Written Exam

In [97]:
R1 = 560
R2 = 1000
Vt = 35
Vr3 = 12
Vr2 = Vt-Vr3
Rt = R2*Vt/Vr2

R3 = Rt - R2
R3

521.7391304347825

In [98]:
Vs = 60
Vr1 = 25
Rt = R1*Vs/Vr1
Rparallel = Rt - R1
Rparallel

784.0

In [99]:
R4 = 1/(1/Rparallel - 1/(R3+R2))
R4

1617.1617161716176

In [100]:
branch = recipsumrecip([R3+R2,R4])

In [101]:
560/sum([R1,branch])*60

25.0

In [102]:
import sympy
from sympy import symbols, Eq

Vx, Vt, Rx, Rt = symbols('Vx,Vt,Rx,Rt')

In [103]:
expr = Eq((Vx/Vt),Rx/Rt)
expr

Eq(Vx/Vt, Rx/Rt)

In [104]:
sympy.solve(expr,Rt)

[Rx*Vt/Vx]

## 2

In [105]:
C1 = .01  *si.µ #Farads
C2 = .22  *si.µ #Farads
C3 = .15  *si.µ #Farads
C4 = .047 *si.µ #Farads
C5 = .01  *si.µ #Farads
C6 = .015 *si.µ #Farads
# C1 + (C2||C3) + (C4 || (C5 + C6))

In [106]:
def recipsumrecip(data):
    """Returns the reciprocal of the sum of reciprocals."""
    denom = 0
    for n in data:
        denom += 1/n
    return 1/denom


# C1 + (C2||C3) + (C4 || (C5 + C6))
c56 = recipsumrecip([C5,C6])
c23 = C2+C3
# C1 + c23 + (C4 || c56)
c456 = C4 + c56
# C1 + c23 + c456
Ctotal = recipsumrecip([C1,c23,c456])
Ctotal

8.225671140939597e-09

In [107]:
def get_capacitive_reactance(capacitance,frequency):
    period = 2*math.pi
    omega = period*frequency
    Xc = 1/(omega*capacitance)
    return complex(0,-Xc)

In [108]:
f = 300 # Hz
Vin = 10

Xc1 = get_capacitive_reactance(C1,f)
Xc2 = get_capacitive_reactance(C2,f)
Xc3 = get_capacitive_reactance(C3,f)
Xc4 = get_capacitive_reactance(C4,f)
Xc5 = get_capacitive_reactance(C5,f)
Xc6 = get_capacitive_reactance(C6,f)

In [109]:
# Sanity check
print(Xc5+Xc6)
print(get_capacitive_reactance(c56,f))

-88419.41282883074j
-88419.41282883075j


In [113]:
Xtotal = get_capacitive_reactance(Ctotal,f)
Itotal = Vin/Xtotal
abs(Xtotal), abs(Itotal)

(64495.22086198853, 0.00015505024816332846)

In [149]:
Ic1 = Itotal            # All current goes through here
Vc1 = Ic1 * Xc1         # Ohm's Law

Vc2 = Vin - Vc1         # All remaining voltage available here
Ic2 = Vc2 / Xc2         # Ohm's Law

Ic3 = Ic1 - Ic2         # Some current splits off, remaing flows through C3
Vc3 = Ic3 * Xc3         # Ohm's Law

Vc4 = Vin - Vc1 - Vc3   # After the voltage drops from C1 and C3, remaing voltage is visible on C4
Ic4 = Vc4 / Xc4         # Ohm's Law

Ic5 = Ic3 - Ic4         # Some current splits off at C4, remaing flows through C5
Ic6 = Ic5               # And also C6, since it's in series with C5
Vc5 = Ic5 * Xc5         # Ohm's Law
Vc6 = Ic6 * Xc6         # Ohm's Law

from matplotlib.ticker import EngFormatter
engFormat = EngFormatter()

print('Ic1 =',engFormat(abs(Ic1)))
print('Vc1 =',engFormat(abs(Vc1)))
print('Ic2 =',engFormat(abs(Ic2)))
print('Vc2 =',engFormat(abs(Vc2)))
print('Ic3 =',engFormat(abs(Ic3))) # I've made a mistake. Needed to treat them as complex numbers
print('Vc3 =',engFormat(abs(Vc3)))
print('Ic4 =',engFormat(abs(Ic4)))
print('Vc4 =',engFormat(abs(Vc4)))
print('Ic5 =',engFormat(abs(Ic5)))
print('Vc5 =',engFormat(abs(Vc5)))
print('Ic6 =',engFormat(abs(Ic6)))
print('Vc6 =',engFormat(abs(Vc6)))

Ic1 = (-0+0.00015505024816332846j)
Vc1 = (8.225671140939596+0j)
Ic2 = (-0+0.0007357968431453012j)
Vc2 = (1.7743288590604038+0j)
Ic3 = -0.0005807465949819728j
Vc3 = (-2.053970917225953-0j)
Ic4 = (-0+0.0003391602283723628j)
Vc4 = (3.828299776286357+0j)
Ic5 = -0.0009199068233543356j
Vc5 = (-48.802572706935166-0j)
Ic6 = -0.0009199068233543356j
Vc6 = (-32.53504847129012-0j)


These numbers seem fishy.  Sanity check time.

In [126]:
# KVL
print( Vin == Vc1 + Vc2 )
print( Vin == Vc1 + Vc3 + Vc4 )
print( Vin == Vc1 + Vc3 + Vc5 + Vc6 )  
# Something is wrong with Vc5 and Vc6, but I've spent too long on this problem. 
# Circle back if I can

print(Itotal , Ic2 + Ic4 + Ic6)

True
True
False
(-0+0.00015505024816332846j) 0.00015505024816332843j


# 3

In [130]:
def get_inductive_reactance(inductance,frequency):
    period = 2*math.pi
    omega = period*frequency
    Xl = omega*inductance
    return complex(0,Xl)

In [137]:
f = 3000
L1 = 5*si.m
L3 = 3*si.m
Vin = 10

Xl1 = get_inductive_reactance(L1,f)
Xl3 = get_inductive_reactance(L3,f)
Il3 = 50*si.m*-1j
Xl1,Xl3,Il3

(94.2477796076938j, 56.548667764616276j, -0.05j)

In [139]:
Vl3 = Il3*Xl3
Vl2 = Vl3
Vl3

(2.827433388230814+0j)

In [142]:
Vl1 = 10 - Vl3
Vl1

(7.172566611769186+0j)

In [144]:
Il1 = Vl1/Xl1
Il1

-0.07610329539459688j

In [145]:
Il2 = Il1 - Il3
Il2

-0.02610329539459688j

In [146]:
Xl2 = Vl2 / Il2
Xl2

(-0+108.31710500491306j)

In [147]:
L2 = Xl2/two_pi(f)
L2

0.0057464008943119305j

# 5

In [152]:
C1 = .033*si.u
f = 10*si.k
Xc1 = get_capacitive_reactance(C1,f)
Xc1

-482.2877063390768j

In [155]:
Itarget = 9*si.m
Vin = 10
Ztotal = Vin/Itarget
Ztotal

1111.111111111111

In [159]:
math.degrees(math.asin(Xc1.imag/Ztotal))

-25.72542760333425

In [163]:
theta = math.asin(Xc1.imag/Ztotal)
R = Ztotal*math.cos(theta)
R

1000.9827518737575

In [164]:
math.sqrt(R**2+Xc1.imag**2)

1111.111111111111

# 8

In [180]:
R = 56
L = 8.2*si.m
C = 18*si.n

fc = 1/two_pi(math.sqrt(L*C))
fc

13100.170645412181

In [181]:
Xl = get_inductive_reactance(L, fc)
Xl

674.9485577105528j

In [182]:
Q = Xl/R
Q = abs(Q)
Q

12.05265281625987

In [183]:
Bw = fc/Q
Bw

1086.9118064812367

In [184]:
delta = Bw/2
upper = fc + delta
lower = fc - delta
lower, upper

(12556.714742171564, 13643.6265486528)